# 2.4: Two-Body Potential Energy Surface

Next, we will parameterize the "physical baseline" ($V_{phys}$) of the PEF.

In this section, we construct, from scratch, a 1-body MB-nrg-style PEF for CH3-NH2.

## 2.4.0 Definitions and Imports

In [ ]:
%run Tutorial_2_Imports.ipynb

In [ ]:
%run Tutorial_2_Paths.ipynb

## 2.4.1 Normal Modes Configuration Generation

In [ ]:
definition_CH3NH2 = read_definition(paths.definitions.systems.CH3NH2)
minimized_structure_CH3NH2, minimized_energy_CH3NH2 = read_system_and_energy(definition_CH3NH2, paths.structures.minimized_CH3NH2)

definition_H2O = read_definition(paths.definitions.systems.H2O)
minimized_structure_H2O, minimized_energy_H2O = read_system_and_energy(definition_H2O, paths.structures.minimized_H2O)

definition_CH3NH2_H2O = read_definition(paths.definitions.systems.CH3NH2_H2O)

In [ ]:
clusters_CH3NH2_H2O = read_systems(definition_CH3NH2_H2O, paths.clusters.CH3NH2_H2O)
vibrational_modes_clusters_CH3NH2_H2O = read_multiple_vibrational_modes(paths.clusters.CH3NH2_H2O_modes)

In [ ]:
configurations_vibrational_modes_CH3NH2_H2O = []

for cluster, modes in zip(clusters_CH3NH2_H2O, vibrational_modes_clusters_CH3NH2_H2O):
    configuration_generator = NormalModesConfigurationGenerator(
            cluster,
            modes,
            distribution="linear",
            temperature=(10, 150)
    )

    configs = list(configuration_generator.generate_configurations(num_configs=150, seed=54321))
    configurations_vibrational_modes_CH3NH2_H2O.extend(configs)
    
write_systems(paths.PIP_CH3NH2_H2O.configs.vibrational_modes, configurations_vibrational_modes_CH3NH2_H2O)

In [ ]:
render_overlayed_systems(
    configurations_vibrational_modes_CH3NH2_H2O,
    centerer=lambda system: system.atoms[0].point,
    aligner=lambda system: (system.atoms[4].point, system.atoms[1].point),
    alpha=0.05,
    num_to_show=100
)

## 2.4.2 Random Distance 

In [ ]:
vibrational_modes_CH3NH2 = read_vibrational_modes(paths.structures.vibrational_modes_CH3NH2)

In [ ]:
vibrational_modes_H2O = read_vibrational_modes(paths.structures.vibrational_modes_H2O)

In [ ]:
generator_CH3NH2 = NormalModesConfigurationGenerator(
        minimized_structure_CH3NH2,
        vibrational_modes_CH3NH2,
)

monomer_configs_CH3NH2 = list(generator_CH3NH2.generate_configurations(num_configs=500, seed=54321))

In [ ]:
generator_H2O = NormalModesConfigurationGenerator(
        minimized_structure_H2O,
        vibrational_modes_H2O,
)

monomer_configs_H2O = list(generator_H2O.generate_configurations(num_configs=500, seed=54321))

In [ ]:
# config_generator = RandomDistanceAndRotationConfigurationGenerator(
#         [monomer_configs_CH3NH2, monomer_configs_H2O],
#         radius=9.0,
#         min_intermolecular_distance_factor=0.8,
#         logarithmic=True,
#         use_pools=True,
#         num_attempts=100
# )

# random_configs_CH3NH2_H2O = list(config_generator.generate_configurations(
#         num_configs=2000,
#         seed=12345,
# ))

# write_systems(paths.PIP_CH3NH2_H2O.configs.random, random_configs_CH3NH2_H2O)
random_configs_CH3NH2_H2O = read_systems(definition_CH3NH2_H2O, paths.PIP_CH3NH2_H2O.configs.random)

In [ ]:
render_overlayed_systems(
    random_configs_CH3NH2_H2O,
    centerer=lambda system: system.atoms[0].point,
    aligner=lambda system: (system.atoms[4].point, system.atoms[1].point),
    alpha=0.10,
    num_to_show=100
)

## 2.4.3 Parallel-Bias Metadynamics Configuration Generation

In [ ]:
bond_params, angle_params, nonbonded_params = get_pbmetad_parameters(
        clusters_CH3NH2_H2O[0]
)

In [ ]:
bond_params

In [ ]:
angle_params

In [ ]:
nonbonded_params

In [ ]:
charges = read_parameters(paths.Vphys.charges)
charges = {symmetry: charges[symmetry]*1/3 for symmetry in charges}
charges["E"] = -0.6*1/3
charges["F"] = +0.3*1/3

In [ ]:
perform_pbmetad_simulation(
        lammps_executable_path,
        [minimized_structure_CH3NH2, minimized_structure_H2O],
        num_configs=500000,
        sample_interval=10,
        temperature=700,
        seed=12345,
        configurations_path=paths.PIP_CH3NH2_H2O.configs.pbmetad_pool,
        bond_params=bond_params,
        angle_params=angle_params,
        nonbonded_params=nonbonded_params,
        charges=charges,
        pbmetad_workdir=paths.scratch.pbmetad_CH3NH2_H2O,
        restart_path=paths.restart.pbmetad_CH3NH2_H2O,
        min_intermolecular_distance=1.5,
        max_intermolecular_distance=9.0
)

In [ ]:
inner_cutoff_2b = 6.0
outer_cutoff_2b = 7.0

In [ ]:
num_filtered_configs = filter_configs_by_cutoff(
        definition_CH3NH2_H2O,
        paths.PIP_CH3NH2_H2O.configs.pbmetad_pool,
        threshold=outer_cutoff_2b,
        filtered_path=paths.PIP_CH3NH2_H2O.configs.pbmetad_pool_filtered
)

In [ ]:
split_pool(
        definition_CH3NH2_H2O,
        paths.PIP_CH3NH2_H2O.configs.pbmetad_pool_filtered,
        pool_size=num_filtered_configs,
        num_configs_to_split=200,
        unsplit_path=paths.PIP_CH3NH2_H2O.configs.pbmetad_train_pool,
        split_path=paths.PIP_CH3NH2_H2O.configs.pbmetad_test,
        strategy="INTERVAL",
)

In [ ]:
pbmetad_configs_CH3NH2_H2O = perform_fps(
        definition_CH3NH2_H2O,
        clusters_CH3NH2_H2O[0],
        vibrational_modes_clusters_CH3NH2_H2O[0],
        paths.PIP_CH3NH2_H2O.configs.pbmetad_train_pool,
        num_pool_configs=num_filtered_configs-200,
        num_phase1_input_configs=100000,
        approx_configs_to_select=2000,
        fps_workdir=paths.scratch.fps_CH3NH2_H2O,
        restart_path=paths.restart.fps_CH3NH2_H2O,
        num_threads=16
)

In [ ]:
render_overlayed_systems(
    pbmetad_configs_CH3NH2_H2O,
    centerer=lambda system: system.atoms[0].point,
    aligner=lambda system: (system.atoms[4].point, system.atoms[3].point),
    alpha=0.10,
    num_to_show=100
)

In [ ]:
write_systems(paths.PIP_CH3NH2_H2O.configs.pbmetad_train, pbmetad_configs_CH3NH2_H2O)

## 2.4.4 Energy Calculation

In [ ]:
calculator = Psi4Calculator(
        "MP2",
        "def2-TZVP",
        log_directory=str(paths.logs),
        scratch_directory=str(paths.scratch.psi4)
)

In [ ]:
energies_vibrational_modes_CH3NH2_H2O = calculate_nmer_energies(
        configurations_vibrational_modes_CH3NH2_H2O,
        calculator,
        paths.restart.energies_vibrational_mode_configs_CH3NH2_H2O,
        num_threads=8,
        mem_mb=8000,
        error_on_any_failure=False,
        retrieve_only=True
)
write_systems_and_nmer_energies(paths.PIP_CH3NH2_H2O.configs.vibrational_modes_energies, (configurations_vibrational_modes_CH3NH2_H2O, energies_vibrational_modes_CH3NH2_H2O))

In [ ]:
energies_pbmetad_CH3NH2_H2O = calculate_nmer_energies(
        pbmetad_configs_CH3NH2_H2O,
        calculator,
        paths.restart.energies_pbmetad_configs_CH3NH2_H2O,
        num_threads=8,
        mem_mb=8000,
        error_on_any_failure=False,
        retrieve_only=True
)
write_systems_and_nmer_energies(paths.PIP_CH3NH2_H2O.configs.pbmetad_train_energies, (pbmetad_configs_CH3NH2_H2O, energies_pbmetad_CH3NH2_H2O))

In [ ]:
energies_random_CH3NH2_H2O = calculate_nmer_energies(
        random_configs_CH3NH2_H2O,
        calculator,
        paths.restart.energies_random_configs_CH3NH2_H2O,
        num_threads=16,
        mem_mb=16000,
        error_on_any_failure=False,
        retrieve_only=True
)
write_systems_and_nmer_energies(paths.PIP_CH3NH2_H2O.configs.random_energies, (random_configs_CH3NH2_H2O, energies_random_CH3NH2_H2O))

In [ ]:
configurations_vibrational_modes_CH3NH2_H2O, energies_vibrational_modes_CH3NH2_H2O = discard_failed_configurations(
        configurations_vibrational_modes_CH3NH2_H2O,
        energies_vibrational_modes_CH3NH2_H2O
)

pbmetad_configs_CH3NH2_H2O, energies_pbmetad_CH3NH2_H2O = discard_failed_configurations(
        pbmetad_configs_CH3NH2_H2O,
        energies_pbmetad_CH3NH2_H2O
)

random_configs_CH3NH2_H2O, energies_random_CH3NH2_H2O = discard_failed_configurations(
        random_configs_CH3NH2_H2O,
        energies_random_CH3NH2_H2O
)

In [ ]:
energies_2b_vibrational_modes_CH3NH2_H2O = calculate_many_body_energies(
    configurations_vibrational_modes_CH3NH2_H2O,
    energies_vibrational_modes_CH3NH2_H2O,
    minimized_monomer_energies=[minimized_energy_CH3NH2, minimized_energy_H2O]
)

energies_2b_pbmetad_CH3NH2_H2O = calculate_many_body_energies(
    pbmetad_configs_CH3NH2_H2O,
    energies_pbmetad_CH3NH2_H2O,
    minimized_monomer_energies=[minimized_energy_CH3NH2, minimized_energy_H2O]
)

energies_2b_random_CH3NH2_H2O = calculate_many_body_energies(
    random_configs_CH3NH2_H2O,
    energies_random_CH3NH2_H2O,
    minimized_monomer_energies=[minimized_energy_CH3NH2, minimized_energy_H2O]
)

In [ ]:
energies_binding_vibrational_modes_CH3NH2_H2O = calculate_binding_energies(
    configurations_vibrational_modes_CH3NH2_H2O,
    energies_vibrational_modes_CH3NH2_H2O,
    minimized_monomer_energies=[minimized_energy_CH3NH2, minimized_energy_H2O]
)

energies_binding_pbmetad_CH3NH2_H2O = calculate_binding_energies(
    pbmetad_configs_CH3NH2_H2O,
    energies_pbmetad_CH3NH2_H2O,
    minimized_monomer_energies=[minimized_energy_CH3NH2, minimized_energy_H2O]
)

energies_binding_random_CH3NH2_H2O = calculate_binding_energies(
    random_configs_CH3NH2_H2O,
    energies_random_CH3NH2_H2O,
    minimized_monomer_energies=[minimized_energy_CH3NH2, minimized_energy_H2O]
)

In [ ]:
write_training_set_file(
        paths.PIP_CH3NH2_H2O.training_set,
        (
                [*configurations_vibrational_modes_CH3NH2_H2O, *pbmetad_configs_CH3NH2_H2O, *random_configs_CH3NH2_H2O],
                [*energies_binding_vibrational_modes_CH3NH2_H2O, *energies_binding_pbmetad_CH3NH2_H2O, *energies_binding_random_CH3NH2_H2O],
                [*energies_2b_vibrational_modes_CH3NH2_H2O, *energies_2b_pbmetad_CH3NH2_H2O, *energies_2b_random_CH3NH2_H2O],
        )
)
print(f"Total configurations for training set: {len(configurations_vibrational_modes_CH3NH2_H2O) + len(pbmetad_configs_CH3NH2_H2O) + len(random_configs_CH3NH2_H2O)}")

## 2.4.5 Polynomial Generation

In [ ]:
pip_2b_max_order = 3

In [ ]:
generate_polynomial(
        definition_CH3NH2_H2O,
        max_degree=pip_2b_max_order,
        polynomial_info_path=paths.PIP_CH3NH2_H2O.PIP.info,
        polynomial_dir=paths.PIP_CH3NH2_H2O.PIP.polynomial,
        filters=[
                "add_filter['sum-degree', 'x-inter-*+*-*', '0']",
                "add_filter['sum-degree', 'x-intra-*+*-*', '2+']",
        ]
)

## 2.4.6 Polynomial Parametrization

In [ ]:
polarizabilities = read_parameters(paths.Vphys.polarizabilities)
polarizability_factors = read_parameters(paths.Vphys.polarizability_factors)
charges = read_parameters(paths.Vphys.charges)
C6 = read_parameters(paths.Vphys.C6)
d6 = read_parameters(paths.Vphys.d6)

In [ ]:
print("polarizabilities:", polarizabilities)
print("polarizability_factors:", polarizability_factors)
print("charges:", charges)
print("C6:", C6)
print("d6:", d6)

In [ ]:
generate_pip_parametizer(
        definition_CH3NH2_H2O,
        max_degree=pip_2b_max_order,
        polynomial_info_path=paths.PIP_CH3NH2_H2O.PIP.info,
        polynomial_dir=paths.PIP_CH3NH2_H2O.PIP.polynomial,
        parametizer_dir=paths.PIP_CH3NH2_H2O.PIP.parameterizer,
        workdir=paths.scratch.PIP_CH3NH2_H2O,
        symmetry_to_charge=charges,
        symmetry_to_polarizability=polarizabilities,
        symmetry_to_polarizability_factor=polarizability_factors,
        symmetries_to_C6=C6,
        symmetries_to_d6=d6,
        monomer_names=["ch3nh2", "h2o"],
        is_mbpol=[False, True],
        inner_cutoff=inner_cutoff_2b,
        outer_cutoff=outer_cutoff_2b
)

In [ ]:
parametrize_pip(
        definition_CH3NH2_H2O,
        parametizer_dir=paths.PIP_CH3NH2_H2O.PIP.parameterizer,
        parametrization_dir=paths.PIP_CH3NH2_H2O.PIP.parametrization,
        training_set_path=paths.PIP_CH3NH2_H2O.training_set,
        workdir=paths.scratch.fitting_CH3NH2_H2O,
        monomer_names=["ch3nh2", "h2o"],
        is_mbpol=[False, True],
        num_fits_to_run=1,
        num_threads=16,
        DE=10,
        alpha=0.0005
)

## 2.4.7 Implementation in MBX

In [ ]:
add_PIP_to_MBX(
        paths.MBX,
        definition_CH3NH2_H2O,
        ["ch3nh2", "h2o"],
        paths.PIP_CH3NH2_H2O.PIP.parameterizer,
        paths.PIP_CH3NH2_H2O.PIP.parametrization,
        max_degree=pip_2b_max_order,
        workdir=paths.scratch.fitting_CH3NH2_H2O,
)

In [ ]:
compile_mbx(
        paths.MBX,
        configure_arguments=[
                "CXX=icpc",
                "CC=icc",
                "--disable-optimization"
        ],
        configure=True,
        clean=False,
        num_threads=4
)

## 2.4.8 Correlation & Validation

In [ ]:
predicted_energies_vibrational_modes_CH3NH2_H2O = calculate_MBX_nmer_energies(
        paths.MBX,
        definition_CH3NH2_H2O,
        configurations_vibrational_modes_CH3NH2_H2O,
        monomer_names=["ch3nh2", "h2o"],
        scratch_dir=paths.scratch.fitting_CH3NH2_H2O
)

In [ ]:
predicted_energies_pbmetad_CH3NH2_H2O = calculate_MBX_nmer_energies(
        paths.MBX,
        definition_CH3NH2_H2O,
        pbmetad_configs_CH3NH2_H2O,
        monomer_names=["ch3nh2", "h2o"],
        scratch_dir=paths.scratch.fitting_CH3NH2_H2O
)

In [ ]:
predicted_energies_random_CH3NH2_H2O = calculate_MBX_nmer_energies(
        paths.MBX,
        definition_CH3NH2_H2O,
        random_configs_CH3NH2_H2O,
        monomer_names=["ch3nh2", "h2o"],
        scratch_dir=paths.scratch.fitting_CH3NH2_H2O
)

In [ ]:
mbx_opt_CH3NH2_energy = calculate_MBX_energy(
        paths.MBX,
        definition_CH3NH2,
        minimized_structure_CH3NH2,
        monomer_names=["ch3nh2"],
        scratch_dir=paths.scratch.fitting_CH3NH2_H2O
)

In [ ]:
mbx_opt_H2O_energy = calculate_MBX_energy(
        paths.MBX,
        definition_H2O,
        minimized_structure_H2O,
        monomer_names=["h2o"],
        scratch_dir=paths.scratch.fitting_CH3NH2_H2O
)

In [ ]:
predicted_energies_2b_vibrational_modes_CH3NH2_H2O = calculate_many_body_energies(
    configurations_vibrational_modes_CH3NH2_H2O,
    predicted_energies_vibrational_modes_CH3NH2_H2O,
    minimized_monomer_energies=[mbx_opt_CH3NH2_energy, mbx_opt_H2O_energy]
)

predicted_energies_2b_pbmetad_CH3NH2_H2O = calculate_many_body_energies(
    pbmetad_configs_CH3NH2_H2O,
    predicted_energies_pbmetad_CH3NH2_H2O,
    minimized_monomer_energies=[mbx_opt_CH3NH2_energy, mbx_opt_H2O_energy]
)

predicted_energies_2b_random_CH3NH2_H2O = calculate_many_body_energies(
    random_configs_CH3NH2_H2O,
    predicted_energies_random_CH3NH2_H2O,
    minimized_monomer_energies=[mbx_opt_CH3NH2_energy, mbx_opt_H2O_energy]
)

In [ ]:
plot_correlation(
    configurations_vibrational_modes_CH3NH2_H2O,
    energies_2b_vibrational_modes_CH3NH2_H2O,
    "MP2",
    {"MBX": predicted_energies_2b_vibrational_modes_CH3NH2_H2O},
    error_width=1.0
)

In [ ]:
plot_correlation(
    configurations_vibrational_modes_CH3NH2_H2O,
    energies_2b_vibrational_modes_CH3NH2_H2O,
    "MP2",
    {"MBX": predicted_energies_2b_vibrational_modes_CH3NH2_H2O},
    error_width=1.0
)

In [ ]:
plot_correlation(
    pbmetad_configs_CH3NH2_H2O,
    energies_2b_pbmetad_CH3NH2_H2O,
    "MP2",
    {"MBX": predicted_energies_2b_pbmetad_CH3NH2_H2O},
    error_width=10.0
)

In [ ]:
plot_correlation(
    random_configs_CH3NH2_H2O,
    energies_2b_random_CH3NH2_H2O,
    "MP2",
    {"MBX": predicted_energies_2b_random_CH3NH2_H2O},
    error_width=1.0
)

In [ ]:
scan_CH3NH2_H2O_1, scan_CH3NH2_H2O_1_energies = read_systems_and_nmer_energies(definition_CH3NH2_H2O, paths.scans.CH3NH2_H2O.first_energies)
scan_CH3NH2_H2O_2, scan_CH3NH2_H2O_2_energies = read_systems_and_nmer_energies(definition_CH3NH2_H2O, paths.scans.CH3NH2_H2O.second_energies)
scan_CH3NH2_H2O_3, scan_CH3NH2_H2O_3_energies = read_systems_and_nmer_energies(definition_CH3NH2_H2O, paths.scans.CH3NH2_H2O.third_energies)

In [ ]:
scan_CH3NH2_H2O_1_2b_energies = calculate_many_body_energies(scan_CH3NH2_H2O_1, scan_CH3NH2_H2O_1_energies)
scan_CH3NH2_H2O_2_2b_energies = calculate_many_body_energies(scan_CH3NH2_H2O_2, scan_CH3NH2_H2O_2_energies)
scan_CH3NH2_H2O_3_2b_energies = calculate_many_body_energies(scan_CH3NH2_H2O_3, scan_CH3NH2_H2O_3_energies)

In [ ]:
predicted_scan_CH3NH2_H2O_1_energies = calculate_MBX_nmer_energies(
        paths.MBX,
        definition_CH3NH2_H2O,
        scan_CH3NH2_H2O_1,
        monomer_names=["ch3nh2", "h2o"],
        scratch_dir=paths.scratch.fitting_CH3NH2_H2O
)
predicted_scan_CH3NH2_H2O_2_energies = calculate_MBX_nmer_energies(
        paths.MBX,
        definition_CH3NH2_H2O,
        scan_CH3NH2_H2O_2,
        monomer_names=["ch3nh2", "h2o"],
        scratch_dir=paths.scratch.fitting_CH3NH2_H2O
)
predicted_scan_CH3NH2_H2O_3_energies = calculate_MBX_nmer_energies(
        paths.MBX,
        definition_CH3NH2_H2O,
        scan_CH3NH2_H2O_3,
        monomer_names=["ch3nh2", "h2o"],
        scratch_dir=paths.scratch.fitting_CH3NH2_H2O
)

In [ ]:
predicted_scan_CH3NH2_H2O_1_2b_energies = calculate_many_body_energies(scan_CH3NH2_H2O_1, predicted_scan_CH3NH2_H2O_1_energies)
predicted_scan_CH3NH2_H2O_2_2b_energies = calculate_many_body_energies(scan_CH3NH2_H2O_2, predicted_scan_CH3NH2_H2O_2_energies)
predicted_scan_CH3NH2_H2O_3_2b_energies = calculate_many_body_energies(scan_CH3NH2_H2O_3, predicted_scan_CH3NH2_H2O_3_energies)

In [ ]:
plot_scan(
        systems=scan_CH3NH2_H2O_1,
        energies={
                "MP2/def2-TZVP": scan_CH3NH2_H2O_1_2b_energies,
                "MBX": predicted_scan_CH3NH2_H2O_1_2b_energies
        },
        atom1_index=4,
        atom2_index=7,
        title="2-body Energies",
        ymax=2,
        centerer=lambda system: system.atoms[4].point,
        aligner=lambda system: (system.atoms[7].point, system.atoms[0].point)
)

In [ ]:
plot_scan(
        systems=scan_CH3NH2_H2O_2,
        energies={
                "MP2/def2-TZVP": scan_CH3NH2_H2O_2_2b_energies,
                "MBX": predicted_scan_CH3NH2_H2O_2_2b_energies
        },
        atom1_index=4,
        atom2_index=7,
        title="2-body Energies",
        ymax=2,
        centerer=lambda system: system.atoms[4].point,
        aligner=lambda system: (system.atoms[7].point, system.atoms[0].point)
)

In [ ]:
plot_scan(
        systems=scan_CH3NH2_H2O_3,
        energies={
                "MP2/def2-TZVP": scan_CH3NH2_H2O_3_2b_energies,
                "MBX": predicted_scan_CH3NH2_H2O_3_2b_energies
        },
        atom1_index=0,
        atom2_index=7,
        title="2-body Energies",
        ymax=2,
        centerer=lambda system: system.atoms[0].point,
        aligner=lambda system: (system.atoms[7].point, system.atoms[4].point)
)

In [ ]:
clusters_CH3NH2_H2O, clusters_CH3NH2_H2O_energies = read_systems_and_nmer_energies(definition_CH3NH2_H2O, paths.clusters.CH3NH2_H2O_energies)

In [ ]:
predicted_clusters_CH3NH2_H2O_energies = calculate_MBX_nmer_energies(
        paths.MBX,
        definition_CH3NH2_H2O,
        clusters_CH3NH2_H2O,
        monomer_names=["ch3nh2", "h2o"],
        scratch_dir=paths.scratch.fitting_CH3NH2_H2O
)

In [ ]:
plot_clusters(
        systems=clusters_CH3NH2_H2O,
        energies={
            "MP2/def2-TZVP": calculate_many_body_energies(clusters_CH3NH2_H2O, clusters_CH3NH2_H2O_energies),
            "MBX": calculate_many_body_energies(clusters_CH3NH2_H2O, predicted_clusters_CH3NH2_H2O_energies)
        },
        title="Cluster 2-body Energies",
        centerer=lambda system: system.atoms[0].point,
        aligner=lambda system: (system.atoms[4].point, system.atoms[1].point),
        ymin=-11,
        render_scale=0.3
)

In [ ]:
definition_CH3NH2_H2O_H2O = read_definition(paths.definitions.systems.CH3NH2_H2O_H2O)

In [ ]:
clusters_CH3NH2_H2O_H2O, clusters_CH3NH2_H2O_H2O_energies = read_systems_and_nmer_energies(definition_CH3NH2_H2O_H2O, paths.clusters.CH3NH2_H2O_H2O_energies)

In [ ]:
predicted_clusters_CH3NH2_H2O_H2O_energies = calculate_MBX_nmer_energies(
        paths.MBX,
        definition_CH3NH2_H2O_H2O,
        clusters_CH3NH2_H2O_H2O,
        monomer_names=["ch3nh2", "h2o", "h2o"],
        scratch_dir=paths.scratch.fitting_CH3NH2_H2O
)

In [ ]:
plot_clusters(
        systems=clusters_CH3NH2_H2O_H2O,
        energies={
            "MP2/def2-TZVP": calculate_many_body_energies(clusters_CH3NH2_H2O_H2O, clusters_CH3NH2_H2O_H2O_energies, nmer_indices=(0, 1)),
            "MBX": calculate_many_body_energies(clusters_CH3NH2_H2O_H2O, predicted_clusters_CH3NH2_H2O_H2O_energies, nmer_indices=(0, 1))
        },
        title="Cluster 2-body Energies",
        centerer=lambda system: system.atoms[0].point,
        aligner=lambda system: (system.atoms[4].point, system.atoms[1].point),
        # ymin=-9,
        render_scale=0.3
)

In [ ]:
definition_CH3NH2_H2O_H2O_H2O = read_definition(paths.definitions.systems.tetramer)

In [ ]:
clusters_CH3NH2_H2O_H2O_H2O, clusters_CH3NH2_H2O_H2O_H2O_energies = read_systems_and_nmer_energies(definition_CH3NH2_H2O_H2O_H2O, paths.clusters.tetramer_energies)

In [ ]:
predicted_clusters_CH3NH2_H2O_H2O_H2O_energies = calculate_MBX_nmer_energies(
        paths.MBX,
        definition_CH3NH2_H2O_H2O_H2O,
        clusters_CH3NH2_H2O_H2O_H2O,
        monomer_names=["ch3nh2", "h2o", "h2o", "h2o"],
        scratch_dir=paths.scratch.fitting_CH3NH2_H2O
)

In [ ]:
plot_clusters(
        systems=clusters_CH3NH2_H2O_H2O_H2O,
        energies={
            "MP2/def2-TZVP": calculate_many_body_energies(clusters_CH3NH2_H2O_H2O_H2O, clusters_CH3NH2_H2O_H2O_H2O_energies, nmer_indices=(0, 1)),
            "MBX": calculate_many_body_energies(clusters_CH3NH2_H2O_H2O_H2O, predicted_clusters_CH3NH2_H2O_H2O_H2O_energies, nmer_indices=(0, 1))
        },
        title="Cluster 2-body Energies",
        centerer=lambda system: system.atoms[0].point,
        aligner=lambda system: (system.atoms[4].point, system.atoms[1].point),
        # ymin=-9,
        render_scale=0.3
)